In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, Bidirectional, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import f1_score
import statistics

In [ ]:
#Extract audio and lexcial features as explained in the multimodal target model
#x_audio_data = np.array([i[0] for i in df_list])  # Audio features
x_text_data = np.array([i[1] for i in df_list])   # Text features
y_data = np.array([i[2] for i in df_list])        # Depression labels (binary: 0 for control, 1 for depressed)
unique_clip_ID_No = np.array([i[3] for i in df_list])  # Unique clip identifier
session_number = np.array([i[4] for i in df_list])     # Session number

## Target model - lexical

In [ ]:
# Constants for the model
NUM_FOLDS = 5
GRU_LAYER_WIDTH = 252
N_GRU_LAYERS = 2
INPUT_SHAPE = (333, 768)
BATCH_SIZE = 64
EPOCHS = 200

# Data preparation
speaker_ids = np.unique(session_number)
k_fold = KFold(n_splits=NUM_FOLDS, shuffle=True)
training_index, testing_index, session_list = [], [], []
accuracies_best, loss_fold_best, F1_best = [], [], []

# Define a custom early stopping callback to restore the best weights
class CustomEarlyStopping(Callback):
    def __init__(self, patience=0, verbose=1):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.verbose = verbose
        self.best_weights = None
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('val_loss')
        if current_loss < self.best:
            self.best = current_loss
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)
                if self.verbose:
                    print(f'Restoring model weights from the end of the best epoch {epoch - self.patience}.')

# Split data into folds for cross-validation
for train_idx, test_idx in k_fold.split(speaker_ids):
    training_index.append(np.where(np.isin(session_number, speaker_ids[train_idx])))
    testing_index.append(np.where(np.isin(session_number, speaker_ids[test_idx])))

# Model training and evaluation
for i in range(NUM_FOLDS):
    print(f'Processing Fold {i + 1}')
    x_train, y_train = x_text_data[training_index[i][0]], y_data[training_index[i][0]]
    x_test, y_test = x_text_data[testing_index[i][0]], y_data[testing_index[i][0]]
    session_no = np.unique(session_number[testing_index[i][0]])
    session_list.append(session_no)

    print(f'Test session for this fold: {session_no}')

    # Build the model
    lexical_input = Input(shape=INPUT_SHAPE, name="lexical_input")
    x = lexical_input
    for _ in range(N_GRU_LAYERS):
        x = Bidirectional(GRU(GRU_LAYER_WIDTH, return_sequences=True))(x)
    x = GlobalAveragePooling1D()(x)
    output = Dense(2, activation='softmax')(Dense(128, activation='relu')(x))
    model = Model(inputs=lexical_input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Fit model with early stopping
    early_stopping = CustomEarlyStopping(patience=3)
    model.fit(x_train, y_train, validation_split=0.2, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping])
    loss, accuracy = model.evaluate(x_test, y_test)

    print(f"Test Loss: {loss}")
    print(f"Test Accuracy: {accuracy}")
    accuracies_best.append(accuracy)
    loss_fold_best.append(loss)

    # Predictions and F1 score evaluation
    y_pred = np.array([np.argmax(model.predict(np.array([x]), verbose=0)[0]) for x in x_test])
    F1_score = f1_score(y_test, y_pred)
    F1_best.append(F1_score)
    print(f"F1 Score: {F1_score}")

# Summarize results
print('--- Summary of Results ---')
for test_set, loss, acc, f1 in zip(session_list, loss_fold_best, accuracies_best, F1_best):
    print(f'Test set: {test_set}, Loss: {loss:.4f}, Accuracy: {acc:.4f}, F1 Score: {f1:.4f}')
print(f'Average accuracy per 10 sec chunk: {np.mean(accuracies_best):.4f}, Std Dev: {statistics.stdev(accuracies_best):.4f}')
print(f'Average F1 score per 10 sec chunk: {np.mean(F1_best):.4f}, Std Dev: {statistics.stdev(F1_best):.4f}')
